#### Lecture 4: 分组

分组操作是数据处理与分析中最为频繁的操作之一。日常生活中也经常遇到分组，比如，按照性别统计全国人口寿命的平均值、按照班级统计某门课的平均成绩、按季度对每月商品的销量进行组内标准化等。

In [ ]:
import numpy as np
import pandas as pd

#### 

##### 一. 分组模式及其对象

1. 分组的一般模式：一个分组操作能够被分割成3个部分：
* 分组的依据：如性别、季节、班级、地区等
* 数据来源： 如全国人口寿命、全年级学生的数学分数
* 具体操作：如求均值、组内标准化、筛选出符合某个条件的组。

分组操作代码的一般模式为

df.groupby(分组依据)[数据来源].具体操作

In [ ]:
# 例1： 按照性别统计身高中位数

df=pd.read_csv('data/learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

In [ ]:
#例2：按照学校分组统计体重的均值
df.groupby('School')['Weight'].mean()

School
Fudan University                 54.000000
Peking University                55.666667
Shanghai Jiao Tong University    56.442308
Tsinghua University              54.223881
Name: Weight, dtype: float64

思考一：根据0.25分位数和0.75分位数进行分割，将体重分为high、normal和low这三组，统计每组身高的均值。

2.  根据多个变量进行分组。只需要在groupby中传入由相应列名构成的列表。

例如，根据学校和性别进行分组，统计学生身高的均值

In [ ]:
df.groupby(['School','Gender'])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

3. 也可以按照给定的条件进行分组。

例如：根据学生体重是否超过总体均值分组，并计算组内的平均身高。

In [ ]:
condition=df.Weight>df.Weight.mean()
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

4. groupby对象上的方法和属性

In [ ]:
gb=df.groupby(['School','Gender'])
gb

In [ ]:
#通过ngroups属性可以得到分组个数
gb.ngroups


8

In [ ]:
#通过groups属性可以返回从组名映射到组索引列表的字典
gb.groups

{('Fudan University', 'Female'): [3, 15, 26, 28, 37, 39, 46, 49, 52, 63, 68, 70, 77, 84, 90, 105, 107, 108, 112, 129, 138, 144, 145, 157, 170, 173, 186, 187, 189, 195], ('Fudan University', 'Male'): [4, 41, 48, 66, 73, 82, 98, 131, 135, 152], ('Peking University', 'Female'): [9, 20, 29, 30, 32, 45, 57, 59, 75, 83, 86, 88, 96, 101, 120, 130, 132, 140, 159, 183, 185, 194], ('Peking University', 'Male'): [1, 35, 36, 38, 54, 61, 72, 99, 102, 116, 127, 147], ('Shanghai Jiao Tong University', 'Female'): [0, 6, 12, 13, 19, 22, 31, 42, 56, 58, 64, 65, 79, 85, 87, 89, 93, 103, 104, 109, 114, 115, 119, 121, 122, 123, 124, 141, 143, 148, 149, 155, 156, 161, 164, 166, 167, 172, 174, 188, 197], ('Shanghai Jiao Tong University', 'Male'): [2, 10, 21, 23, 50, 60, 71, 117, 134, 153, 165, 171, 184, 190, 192, 198], ('Tsinghua University', 'Female'): [5, 7, 8, 11, 14, 25, 27, 33, 34, 43, 44, 47, 51, 53, 55, 62, 67, 69, 78, 80, 81, 92, 97, 100, 106, 110, 111, 113, 118, 125, 126, 128, 133, 136, 137, 139, 14

In [ ]:
#通过size属性可以获得每个组的元素个数
gb.size()

School                         Gender
Fudan University               Female    30
                               Male      10
Peking University              Female    22
                               Male      12
Shanghai Jiao Tong University  Female    41
                               Male      16
Tsinghua University            Female    48
                               Male      21
dtype: int64

In [ ]:
#通过get_group()方法可以直接获取元素所在组对应的行。需要指定组的具体名字
gb.get_group(('Fudan University',''))

其他的属性方法，包括mean()、median（）等。

##### 二. 聚合函数

1. groupby对象上的内置聚合函数，

包括max()、min()、mean()、media()、count()、all()、any()、idxmax()、idxmin()、unique()、skew()、quantile()、sum()、std()、var()等

In [ ]:
gd=df.groupby('Gender')['Height']
gd.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [18]:
gd.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

当传入的数据包含多个列时，这些聚合函数将按照列进行迭代计算

In [19]:
gd=df.groupby('Gender')[['Height','Weight']]
gd.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


思考2：在learn_pandas.csv数据集中，Transfer列元素为“N”时表示该同学不是转系生，请按照学校和年级两列分组，找出所有不含转系生的组对应的学校和年级

2. agg()函数

虽然groupby对象上实现了许多内置函数，但是单纯依靠这些内置函数扔难以满足应用需要，包括：

* 同时使用多个函数
* 对特定的列使用特定的聚合函数
* 使用自定义的聚合函数
* 直接对结果的列名在聚合前进行自定义命名

使用agg()函数可以解决这四类问题

a) 使用多个函数：需要用列表的形式把内置聚合函数对应的字符串传入到agg()中。

In [20]:
gb.agg(['sum','idxmax','skew'])

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_51562/700879022.py:1: FutureWarning: ['Grade', 'Name', 'Transfer', 'Test_Date', 'Time_Record'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  gb.agg(['sum','idxmax','skew'])


Height                   Weight         \
                                         sum idxmax      skew     sum idxmax   
School                        Gender                                           
Fudan University              Female  4128.2     28  0.093769  1437.0     28   
                              Male    1393.7     48 -1.169826   723.0     66   
Peking University             Female  3332.0     75 -0.174257   933.0     75   
                              Male    1720.3     38  0.702021   737.0     38   
Shanghai Jiao Tong University Female  6364.9     64 -0.405679  1795.0     64   
                              Male    2651.4      2  0.153731  1140.0      2   
Tsinghua University           Female  7188.9     55 -0.169096  2304.0     14   
                              Male    3089.5    193  0.974132  1329.0     40   

                                               Test_Number                   
                                          skew         sum idxmax      skew  
School                        Gender                                         
Fudan University              Female  0.031699          53     15  0.201417  
                              Male   -0.444561          15    152  1.178511  
Peking University             Female -0.086448          38      9  0.528977  
                              Male   -0.236827          19     38  0.987605  
Shanghai Jiao Tong University Female -0.698009          72     19  0.478381  
                              Male   -0.065481          23     50  1.183116  
Tsinghua University           Female -0.323913          73     33  0.963331  
                              Male   -1.334529          36     76  0.576261

b) 对特定的列使用特定的聚合函数：聚合函数和对应的列通过字典的形式传递给agg()，字典以列名为键，以聚合函数字符串或字符串列表为值。

In [21]:
gd.agg({'Height':['mean','max'],'Weight':'count'})

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

c)使用自定义函数：在agg()中传入自定义的函数。传入函数的参数是之前数据源中的列，逐列进行计算。

In [22]:
#计算身高和体重的极差
gb.agg(lambda x:x.max()-x.min())

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_51562/471073041.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  gb.agg(lambda x:x.max()-x.min())


Height  Weight  Test_Number
School                        Gender                             
Fudan University              Female    22.9    29.0            2
                              Male       9.7    19.0            2
Peking University             Female    22.2    22.0            2
                              Male      22.9    29.0            2
Shanghai Jiao Tong University Female    22.3    23.0            2
                              Male      22.9    27.0            2
Tsinghua University           Female    18.4    21.0            2
                              Male      38.2    28.0            2

In [23]:
def my_func(s):
    res='High'
    if s.mean()<=df[s.name].mean():
        res='Low'
    return res
gb.agg(my_func)

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_51562/42475160.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  gb.agg(my_func)


Height Weight Test_Number
School                        Gender                          
Fudan University              Female    Low    Low        High
                              Male     High   High         Low
Peking University             Female    Low    Low        High
                              Male     High   High         Low
Shanghai Jiao Tong University Female    Low    Low        High
                              Male     High   High         Low
Tsinghua University           Female    Low    Low         Low
                              Male     High   High        High

4. 对聚合结果重命名

只需要将聚合函数处改成元组，元组的第一个元素为新的名字，第二个元素为原来的聚合函数。

In [24]:
gd.agg([('range',lambda x:x.max()-x.min()),('my_sum','sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

##### 三、变换函数

变换函数的返回值为与数据源同长度的序列，不会改变原表中的序列。最常用的内置变换函数是累积函数：cumcount()、cumsum()、cumprod()、cummax()、cummin()。使用方法与聚合函数类似，只不过完成的是组内累计。

In [25]:
example=pd.DataFrame({'A':list('aaabba'),'B':[3,6,5,2,1,7]})
example

,A,B
0,a,3
1,a,6
2,a,5
3,b,2
4,b,1
5,a,7


In [26]:
example.groupby('A')['B'].cumcount()

0    0
1    1
2    2
3    0
4    1
5    3
dtype: int64

In [27]:
#对某一列按照元素的取值进行连续编号
example.groupby('A')['A'].cumcount()

0    0
1    1
2    2
3    0
4    1
5    3
dtype: int64

也可以进行自定义变换，使用transform()函数。自定义函数的传入值为数据源的序列，返回的结果是行索引与数据源一直的Series或者DataFrame

例如：对身高和体重进行分组标准化处理，即减去组均值后除以组的标准差

In [28]:
gd.transform(lambda x:(x-x.mean())/x.std())

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631
...,...,...
195,-1.048078,-0.354888
196,0.336968,0.385033
197,-1.048078,-0.539868
198,0.237570,-0.226342


##### 四、跨列分组

有一些常见的分组无法用前面介绍的任何一种方法处理。例如：定义身体质量指数（BMI）=weight/height^2。

体重和审稿的单位分别为千克和米，需要分组计算组BMI的均值

在groupby对象上使用apply()

In [29]:
def BMI(x):
    Height=x['Height']/100
    Weight=x['Weight']
    BMI_value=Weight/Height**2
    return BMI_value.mean()
gb.apply(BMI)

School                         Gender
Fudan University               Female    18.891092
                               Male      24.401886
Peking University              Female    18.473946
                               Male      24.629279
Shanghai Jiao Tong University  Female    19.106968
                               Male      24.522800
Tsinghua University            Female    18.810067
                               Male      23.946918
dtype: float64

除了返回标量，apply()函数还可以返回一维Series和二维DataFrame。

In [31]:
# 1. 标量的情况：无论数据源是单列或多列，此时得到的结果是Series，索引与agg()的结果一致。
gb1=df.groupby(['Gender','Test_Number'])['Height']
gb2=df.groupby(['Gender','Test_Number'])[['Height','Weight']]

gb1.apply(lambda x:0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
Name: Height, dtype: int64

In [32]:
# 也可以返回列表，
gb2.apply(lambda x:[0,0]) #注意，此时吧数组当做标量看待。

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

In [34]:
#2. Seriesq情况：当数据源为单列时，得到的是Series，原来的行索引会被加到新表的最内层
gb1.apply(lambda x: pd.Series([0,0],index=['a','b']))

Gender  Test_Number   
Female  1            a    0
                     b    0
        2            a    0
                     b    0
        3            a    0
                     b    0
Male    1            a    0
                     b    0
        2            a    0
                     b    0
        3            a    0
                     b    0
Name: Height, dtype: int64

In [35]:
# 当数据源为多列时，得到的是DataFrame，原来的行索引会当做新表的列索引

gb2.apply(lambda x:pd.Series([0,0],index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

In [40]:
#3. DataFrame情况

# 无论是数据源是单列还是多列，得到的结果都是DataFrame，原来的行索引会被添加到新表的最内层，原来的列索引会作为新表的列索引
temp_df=pd.DataFrame(np.ones((2,2)),index=['a','b'],columns=pd.Index([('w','x'),('y','z')]))
temp_df.head()


,w,y
,x,z
a,1.0,1.0
b,1.0,1.0


In [41]:
gb1.apply(lambda x:temp_df).head()

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0

In [42]:
gb2.apply(lambda x:temp_df).head()

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0

#### 练习

1. 汽车数据分组分析

现有一汽车数据集，car.csv，其中Brand、Disp和HP分别代表汽车品牌和发动机蓄量、发动机输出功率。

(1)按照如下要求，逐步对表格数据进行操作
* 筛选出所属Country数超过2的汽车，即若该汽车的Country在总体数据集中出现次数不超过2则剔除
* 再按Country分组计算价格均值、价格变异系数和该Country的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为Cov

(2) 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计Price的均值

(3) 按照Type分组，解决以下问题：
* 对Price和HP分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单级索引
* 对HP进行组内的min-max归一化，即每个元素减去组内HP的最小值后，再除以组内HP的极差

2. 某海洋物种在三大海域的分布研究

2001年1月-2020年12月，某科研团队对某海洋物种在太平洋部分水域（西径120~160，赤道线~南维40）、印度洋水域（东经60~100，赤道线~南维40）和大西洋部分水域（0经线~西径40、南维20~60）的出现情况进行了记录。记录的数据表存储在marine_observation.csv中，表中的每一行数据包含了该次观测的时间、经纬度坐标以及海水盐度

(1) 分组计算各年份在各海域的观测次数与海水盐度均值

(2) 逐月统计每个水域的观测总次数